In [79]:
!date

Sat Feb 15 10:44:08 UTC 2020


# Kubernetes操作

## Kubernetesリソースの作成

### GitHubからYAMLを取得する
プロジェクトのGitHubからGKEでWordPressを動かすためのYAMLファイルを取得する。

In [80]:
branch_name="master"
!rm -rf k8s-wordpress-mysql
!git clone -b {branch_name} https://github.com/topselc4ri/k8s-wordpress-mysql.git

Cloning into 'k8s-wordpress-mysql'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 92 (delta 45), reused 53 (delta 21), pack-reused 0
Unpacking objects: 100% (92/92), done.


### [オプション] kubectlの設定
GKEクラスタの構築から続けて実行する場合は、この手順を省略可能。

In [81]:
cluster_name = "topse-cluster"
region="us-west1"
!gcloud container clusters get-credentials {cluster_name} --region {region}
!curl -LO https://storage.googleapis.com/kubernetes-release/release/$(curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt)/bin/linux/amd64/kubectl
!chmod +x ./kubectl
!sudo mv ./kubectl /usr/local/bin/kubectl
!kubectl version 

Fetching cluster endpoint and auth data.
kubeconfig entry generated for topse-cluster.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.4M  100 41.4M    0     0   119M      0 --:--:-- --:--:-- --:--:--  119M
Client Version: version.Info{Major:"1", Minor:"17", GitVersion:"v1.17.3", GitCommit:"06ad960bfd03b39c8310aaf92d1e7c12ce618213", GitTreeState:"clean", BuildDate:"2020-02-11T18:14:22Z", GoVersion:"go1.13.6", Compiler:"gc", Platform:"linux/amd64"}
Server Version: version.Info{Major:"1", Minor:"14+", GitVersion:"v1.14.10-gke.17", GitCommit:"bdceba0734835c6cb1acbd1c447caf17d8613b44", GitTreeState:"clean", BuildDate:"2020-01-17T23:10:13Z", GoVersion:"go1.12.12b4", Compiler:"gc", Platform:"linux/amd64"}


## persistentvolume等のデプロイ

In [82]:
!cat k8s-wordpress-mysql/nfs/gcp-filestore.yaml
!cat k8s-wordpress-mysql/nfs/persistent-volume-claim.yaml

apiVersion: v1
kind: PersistentVolume
metadata:
  name: fileserver
spec:
  capacity:
    storage: 1G
  accessModes:
  - ReadWriteMany
  nfs:
    path: /vol1
    server: nfs.topse.private

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: fileserver-claim
spec:
  accessModes:
  - ReadWriteMany
  # It's necessary to specify "" as the storageClassName
  # so that the default storage class won't be used, see
  # https://kubernetes.io/docs/concepts/storage/persistent-volumes/#class-1
  storageClassName: ""
  resources:
    requests:
      storage: 100M



In [83]:
!kubectl apply -f k8s-wordpress-mysql/nfs/gcp-filestore.yaml
!kubectl apply -f k8s-wordpress-mysql/nfs/persistent-volume-claim.yaml

persistentvolume/fileserver created
persistentvolumeclaim/fileserver-claim created


In [84]:
!kubectl get PersistentVolume

NAME         CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                      STORAGECLASS   REASON   AGE
fileserver   1G         RWX            Retain           Bound    default/fileserver-claim                           3s


## wordpressのDeplyomentのデプロイ

In [85]:
!cat k8s-wordpress-mysql/nfs/gcp-filestore.yaml
!cat k8s-wordpress-mysql/nfs/persistent-volume-claim.yaml

apiVersion: v1
kind: PersistentVolume
metadata:
  name: fileserver
spec:
  capacity:
    storage: 1G
  accessModes:
  - ReadWriteMany
  nfs:
    path: /vol1
    server: nfs.topse.private

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: fileserver-claim
spec:
  accessModes:
  - ReadWriteMany
  # It's necessary to specify "" as the storageClassName
  # so that the default storage class won't be used, see
  # https://kubernetes.io/docs/concepts/storage/persistent-volumes/#class-1
  storageClassName: ""
  resources:
    requests:
      storage: 100M



In [86]:
!kubectl apply -f k8s-wordpress-mysql/nfs/gcp-filestore.yaml
!kubectl apply -f k8s-wordpress-mysql/nfs/persistent-volume-claim.yaml

persistentvolume/fileserver unchanged
persistentvolumeclaim/fileserver-claim unchanged


In [87]:
!kubectl get PersistentVolumeClaim

NAME               STATUS   VOLUME       CAPACITY   ACCESS MODES   STORAGECLASS   AGE
fileserver-claim   Bound    fileserver   1G         RWX                           7s


## serviceのデプロイ

In [88]:
!cat k8s-wordpress-mysql/wordpress/wordpress.yaml

apiVersion: apps/v1

kind: Deployment
metadata:
  name: wordpress
spec:
  replicas: 2
  selector:
    matchLabels:
      app: wordpress
  template:
    metadata:
      labels:
        app: wordpress
    spec:
      containers:
        - image: wordpress:4.8-apache
          name: wordpress
          env:
          - name: WORDPRESS_DB_HOST
            value: mysql.topse.private:3306
          - name: WORDPRESS_DB_PASSWORD
            value: 8sg3jJguHLq8KjNk
          ports:
            - containerPort: 80
              name: wordpress
          readinessProbe:
                  httpGet:
                          path: /readme.html
                          port: 80
                  initialDelaySeconds: 5
                  periodSeconds: 10
          volumeMounts:
            - mountPath: /var/www/html/wp-content/uploads
              name: mypvc
          lifecycle:
            postStart:
              exec:
                command: ["/bin/sh", "-c", "chown -R www-data:www-data /var/w

In [89]:
!kubectl apply -f k8s-wordpress-mysql/wordpress/wordpress.yaml

deployment.apps/wordpress created


In [90]:
!kubectl get pods

NAME                         READY   STATUS              RESTARTS   AGE
wordpress-7b6867cc99-9rkrz   0/1     ContainerCreating   0          1s
wordpress-7b6867cc99-p25fb   0/1     ContainerCreating   0          1s


## ingressのデプロイ

ingressにパブリックIPアドレス(ADDRESS)が割り当てられるまでには5分程かかります。
更に5分～10分程待つと、WebブラウザからアクセスしてWordPressの画面が見えるようになる。

In [91]:
!cat k8s-wordpress-mysql/wordpress/wordpress-backendconfig.yaml
!cat k8s-wordpress-mysql/wordpress/wordpress-service.yaml
!cat k8s-wordpress-mysql/wordpress/wordpress-ingress.yaml

apiVersion: cloud.google.com/v1beta1
kind: BackendConfig
metadata:
  name: wordpress-backendconfig
spec:
  timeoutSec: 40
  connectionDraining:
    drainingTimeoutSec: 60
  sessionAffinity:
    affinityType: "GENERATED_COOKIE"
    affinityCookieTtlSec: 50
apiVersion: v1
kind: Service
metadata:
  name: wordpress
  annotations:
          cloud.google.com/neg: '{"ingress": true}'
          beta.cloud.google.com/backend-config: '{"ports": {"80":"wordpress-backendconfig"}}'
spec:
  type: NodePort
  ports:
  - port: 80
    protocol: TCP
    targetPort: 80
  selector:
    app: wordpress
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: wordpress-ingress
spec:
  rules:
  - http:
      paths:
      - backend:
          serviceName: wordpress
          servicePort: 80


In [92]:
!kubectl apply -f k8s-wordpress-mysql/wordpress/wordpress-backendconfig.yaml
!kubectl apply -f k8s-wordpress-mysql/wordpress/wordpress-service.yaml
!kubectl apply -f k8s-wordpress-mysql/wordpress/wordpress-ingress.yaml

backendconfig.cloud.google.com/wordpress-backendconfig created
service/wordpress created
ingress.extensions/wordpress-ingress created


In [93]:
!kubectl get backendconfig

NAME                      AGE
wordpress-backendconfig   4s


In [94]:
!kubectl get service

NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)        AGE
kubernetes   ClusterIP   10.56.0.1    <none>        443/TCP        91m
wordpress    NodePort    10.56.6.86   <none>        80:31178/TCP   4s


In [95]:
!kubectl get ingress

NAME                HOSTS   ADDRESS   PORTS   AGE
wordpress-ingress   *                 80      4s


Ingress作成待ち

In [97]:
address = ""
while len(address) < 1:
    address = !kubectl get ingress -o jsonpath --template {.items[0].status.loadBalancer.ingress[0].ip}
    print("waiting...")
    time.sleep(10)
print(address)

waiting...
waiting...
['35.190.117.3']


ロードバランサ準備待ち

In [98]:
import requests
import time

while True:
    
    try:
        r = requests.get('http://'+ address[0])
    except ConnectionError:
        print("not establish")

    if r.status_code < 400 :
        print(r.status_code)
        break
        
    print("waiting... (status_code=" + str(r.status_code) + ")")
    time.sleep(10)

!curl -I {"http://" + address[0]}

waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=404)
waiting... (status_code=502)
200
HTTP/1.1 302 Found
Date: Sat, 15 Feb 2020 10:49:09 GMT
Server: Apache/2.4.10 (Debian)
X-Powered-By: PHP/5.6.32
Expires: Wed, 11 Jan 1984 05:00:00 GMT
Cache-Control: no-cache, must-revalidate, max-age=0
Location: http://35.190.117.3/wp-admin/install.php
Content-Type: text/html; charset=UTF-8
Via: 1.1 google
Set-Cookie: GCLB=CMCt7Nu4wKCwDg; path=/; HttpOnly; expires=Sat, 15-Feb-2020 10:49:59 GMT
Transfer-Encoding: chunked



In [99]:
!date

Sat Feb 15 10:49:10 UTC 2020
